1. Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from opacus import PrivacyEngine
import copy

2. Setup and Criterion Definition

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()

3. Model Definition

In [ ]:
model_dp = SimpleCNN().to(device)
optimizer = optim.Adam(model_dp.parameters(), lr=1e-3)

4. Addition of DP to Model

In [ ]:
privacy_engine = PrivacyEngine()
model_dp, optimizer, train_loader = privacy_engine.make_private(
    module=model_dp,
    optimizer=optimizer,
    data_loader=train_loader,
    noise_multiplier=1.0,
    max_grad_norm=1.0,
)

5. Model Training

In [ ]:
train(model_dp, train_loader, optimizer, criterion, device, epochs=5)

6. Copy Model and Remove a Single Sample

model_full = copy.deepcopy(model_dp)

subset_indices = list(range(1, len(train_dataset)))
train_subset = torch.utils.data.Subset(train_dataset, subset_indices)
train_loader_subset = torch.utils.data.DataLoader(train_subset, batch_size=64, shuffle=True)


7. Retrain Model with Sample Removed

In [ ]:
model_dp_removed = SimpleCNN().to(device)
optimizer_removed = optim.Adam(model_dp_removed.parameters(), lr=1e-3)
privacy_engine_removed = PrivacyEngine()
model_dp_removed, optimizer_removed, train_loader_removed = privacy_engine_removed.make_private(
    module=model_dp_removed,
    optimizer=optimizer_removed,
    data_loader=train_loader_subset,
    noise_multiplier=1.0,
    max_grad_norm=1.0,
)
train(model_dp_removed, train_loader_removed, optimizer_removed, criterion, device, epochs=5)


8. Compare Outputs

In [ ]:
test_sample, _ = next(iter(test_loader))
test_sample = test_sample.to(device)

output_full = model_full(test_sample)
output_removed = model_dp_removed(test_sample)

difference = torch.abs(output_full - output_removed).mean().item()
print(f"Average difference in outputs with and without one sample: {difference:.6f}")

9. Visualize

In [ ]:
import matplotlib.pyplot as plt

plt.plot(output_full[0].detach().cpu().numpy(), label="Full Dataset")
plt.plot(output_removed[0].detach().cpu().numpy(), label="One Sample Removed", linestyle="--")
plt.legend()
plt.title("Model Output Comparison with DP")
plt.xlabel("Class Index")
plt.ylabel("Prediction Confidence")
plt.show()
